# Non supervised text topic detection

## LDA with Gensim

In [0]:
import os
import re
import string

#import pandas as pd

data_path = '.'

In [0]:
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train',
                 remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
print(len(twenty_train['data']))

In [0]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer

def tokenize_doc(text):
    sents = sent_tokenize(text)
    tokens = []
    for s in sents:
        for t in TreebankWordTokenizer().tokenize(s):
            if len(t)<2 or '\n' in t or '\t' in t or t.isnumeric():
                continue
            else:
                tokens += [t.lower()]
    return tokens

In [0]:
%%time

docs_tokenized = []
for text in twenty_train['data']:
    docs_tokenized += [tokenize_doc(text)]
print(len(docs_tokenized))

In [0]:
# Get the NLTK stopwords list
import nltk

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

In [0]:
# create dictionary and bag of words corpus

from gensim.corpora import Dictionary, MmCorpus
    
print('Building dictionary...')
dictionary = Dictionary(docs_tokenized)

# Stopwords

additional_stopwords=set(['...', '\'s','\'\'',  "``", "n\'t", "\'re", "\'m", "\'ve",
                          '--', '"', "\'ax", 'max', 'q,3'])

#additional_stopwords=set()

stopwords = nltk_stopwords().union(additional_stopwords)
stopword_ids = map(dictionary.token2id.get, stopwords)
dictionary.filter_tokens(stopword_ids)
dictionary.compactify()

# Filter extremes
dictionary.filter_extremes(no_below=5, no_above=0.95, keep_n=None)
dictionary.compactify()

print('Building corpus...')
corpus = [dictionary.doc2bow(doc) for doc in docs_tokenized]

In [0]:
%%time

# Train the LDA model
from gensim import models

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=25, passes=10)
                                      
lda.save(os.path.join(data_path, 'newsgroups_50_lda.model'))


In [0]:
# Print summary of topics
lda.show_topics()


In [0]:
# get_document_topics
lda.get_document_topics(corpus[:10])


In [0]:
# Print topics details
for i in range(lda.num_topics):
    print(i, lda.show_topic(i))
    print('\n')
    

# Visualization

In [0]:
# Install pyLDAvis in colab

! pip install pyldavis


In [0]:
import pyLDAvis

pyLDAvis.enable_notebook()


In [0]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, corpus, dictionary)
